<a href="https://colab.research.google.com/github/Rephonic/STDM_coursework/blob/main/PreProcessData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import seaborn as sns
import numpy as np
from folium.plugins import HeatMap
import folium


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# File is too large, not able to upload to github
# shared link of Google Drive:
#https://drive.google.com/drive/folders/1CDMo5iFs5LIkNssqT6r6uwU1kJB6ii0d?usp=sharing

df = pd.read_csv('/content/drive/MyDrive/STDM/Crimes_10_year.csv')

In [ ]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,13130561,JG329032,01/01/2014 12:00:00 AM,033XX W LEXINGTON ST,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,APARTMENT,False,True,...,24.0,27.0,17,NaN,NaN,2014,07/06/2023 03:48:21 PM,NaN,NaN,NaN
1,10778567,HZ545087,01/01/2014 12:00:00 AM,019XX E 79TH ST,1585,SEX OFFENSE,OTHER,APARTMENT,False,True,...,8.0,43.0,17,NaN,NaN,2014,09/07/2021 03:41:02 PM,NaN,NaN,NaN
2,12357943,JE220843,01/01/2014 12:00:00 AM,026XX W 13TH ST,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,False,True,...,28.0,29.0,17,NaN,NaN,2014,05/06/2021 03:40:12 PM,NaN,NaN,NaN
3,11729541,JC314736,01/01/2014 12:00:00 AM,045XX N HAZEL ST,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,False,False,...,46.0,3.0,11,NaN,NaN,2014,06/21/2019 03:40:42 PM,NaN,NaN,NaN
4,12729039,JF278612,01/01/2014 12:00:00 AM,042XX W MONROE ST,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,RESIDENCE,False,True,...,28.0,26.0,02,NaN,NaN,2014,11/16/2023 03:40:49 PM,NaN,NaN,NaN


In [ ]:
districts = gpd.read_file('poldist.shp')
districts = districts.to_crs('EPSG:4326')

In [ ]:
print(districts.columns)

Index(['DISTNO', 'DISTNAME', 'geometry'], dtype='object')


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['YearMonth'] = df['Date'].dt.to_period('M')

In [ ]:
df['geometry'] = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
geo_df = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')


In [ ]:
top_types = df['Primary Type'].value_counts().nlargest(3).index
top_types

Index(['THEFT', 'BATTERY', 'CRIMINAL DAMAGE'], dtype='object')

In [ ]:
filtered_df = df[df['Primary Type'].isin(top_types)]

In [ ]:
filtered_df['geometry'] = [Point(xy) for xy in zip(filtered_df['Longitude'], filtered_df['Latitude'])]
geo_df = gpd.GeoDataFrame(filtered_df, geometry='geometry', crs='EPSG:4326')

cases_with_district = gpd.sjoin(geo_df, districts, how='left', predicate='within')

monthly_district_type_cases = cases_with_district.groupby(['YearMonth', 'DISTNO', 'Primary Type']).size()

monthly_district_type_cases_df = monthly_district_type_cases.reset_index(name='CaseCount')

all_periods = pd.period_range(filtered_df['Date'].min(), filtered_df['Date'].max(), freq='M')
all_districts = districts['DISTNO'].unique()

idx = pd.MultiIndex.from_product([all_periods, all_districts, top_types], names=['YearMonth', 'DISTNO', 'Primary Type'])
monthly_district_type_cases_df = monthly_district_type_cases_df.set_index(['YearMonth', 'DISTNO', 'Primary Type']).reindex(idx, fill_value=0).reset_index()

<ipython-input-56-e3b494df923f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['geometry'] = [Point(xy) for xy in zip(filtered_df['Longitude'], filtered_df['Latitude'])]


In [ ]:
monthly_district_type_cases_df.to_csv('monthly_3type_long.csv', index=False)

In [ ]:
monthly_district_type_cases_df['DISTNO'] = monthly_district_type_cases_df['DISTNO'].astype(str)

for primary_type in monthly_district_type_cases_df['Primary Type'].unique():
    subset = monthly_district_type_cases_df[monthly_district_type_cases_df['Primary Type'] == primary_type]
    wide_format = subset.pivot(index='YearMonth', columns='DISTNO', values='CaseCount')
    ordered_columns = sorted(wide_format.columns, key=lambda x: (x.isdigit(), int(x) if x.isdigit() else x))
    wide_format = wide_format[ordered_columns]
    output_filename = f'{primary_type}_wide_format.csv'
    wide_format.to_csv(output_filename)

    print(f"Saved {primary_type} data to {output_filename}")

Saved THEFT data to THEFT_wide_format.csv
Saved BATTERY data to BATTERY_wide_format.csv
Saved CRIMINAL DAMAGE data to CRIMINAL DAMAGE_wide_format.csv
